In [ ]:
# quick sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# late fusion training libraries
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
# pipeline and grid search libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# classification report
from sklearn.metrics import classification_report

In [ ]:
# load the csv file
df = pd.read_csv('your csv file')
# split into train and test sets
train, test = train_test_split(df, test_size=0.33)

In [ ]:
# set the X and y columns
x_cols = ['list of column names to use as your features']
y_cols = ['column name of the label']
# set the X and y
X_train = train[x_cols]
y_train = train[y_cols]

X_test = test[x_cols]
y_test = test[y_cols]

In [ ]:
# set up grid search parameters for different models (other models and model parameters can be used see: 
# https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
if fusion_model == 'rf':  # random forest
    pipe = Pipeline([('scaler', StandardScaler()), 
                     ('RF', RandomForestClassifier(random_state=rand_state))])
    params = {'scaler': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
              'RF__n_estimators':[10, 25, 50, 100], 
              'RF__criterion':["gini", "entropy", "log_loss"],
              'RF__class_weight': ['balanced']
             }
elif fusion_model == 'lr': # logistic regression classifier
    pipe = Pipeline([('scaler', StandardScaler()), 
                     ('LR', LogisticRegression(random_state=rand_state))])
    params = {'scaler': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
              'LR__penalty':["l1", "l2", 'elasticnet'], 
              'LR__tol':[1.0e-3, 1.0e-4, 1.0e-5],
              'LR__C': [0.1, 0.25, 0.5, 0.75, 1.0],
              'LR__class_weight': ['balanced'],
              'LR__max_iter': [100, 250, 500, 1000]
             }
else: # SVM
    pipe = Pipeline([('scaler', StandardScaler()), 
                     ('SVM', SVC(probability=True, random_state=rand_state))])
    params = {'scaler': [StandardScaler(), MinMaxScaler(), Normalizer(), MaxAbsScaler()],
              'SVM__C':[0.001,0.1,10,100,10e5], 
              'SVM__gamma':[0.1,0.01]
             }

In [ ]:
# train models through grid search
clf = GridSearchCV(pipe, param_grid=params, cv=5)
clf.fit(X_train, y_train)

In [ ]:
# get the prediction 
Y_pred = clf.predict(X_test) # get prediction label
Y_scores = clf.predict_proba(X_test) # get prediction probability (some sklearn models do not have this option or needs a flag during training to have this option)

In [ ]:
# get classification report
print(classification_report(y_test, Y_pred))